# Option Pricing Project – USD/CNY
**Name:** Jaskaran Singh  
**Entry Number:** 2023MCB1297

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy import stats
from statsmodels.graphics.gofplots import qqplot
import datetime
from numpy.random import normal
from math import exp, sqrt, log
from scipy.stats import norm

In [ ]:
# Download historical data
ticker = 'USDCNY=X'
data = yf.download(ticker, progress=False)
data = data.dropna()
data.to_csv('USDCNY.csv')
data.head()

In [ ]:
# Plot the closing prices
plt.figure(figsize=(10,5))
plt.plot(data['Close'], label='USD/CNY Exchange Rate')
plt.title('USD/CNY Exchange Rate')
plt.xlabel('Date')
plt.ylabel('Exchange Rate')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compute log returns
data['LogReturn'] = np.log(data['Close'] / data['Close'].shift(1))
log_returns = data['LogReturn'].dropna()

# Plot histogram and QQ plot
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.histplot(log_returns, kde=True)
plt.title('Histogram of Log Returns')
plt.subplot(1,2,2)
qqplot(log_returns, line='s')
plt.title('QQ Plot of Log Returns')
plt.tight_layout()
plt.show()

# Statistical tests
jb_test = stats.jarque_bera(log_returns)
ks_test = stats.kstest(log_returns, 'norm', args=(log_returns.mean(), log_returns.std()))
ad_test = stats.anderson(log_returns, dist='norm')

jb_test, ks_test, ad_test

In [ ]:
# Annualized historical volatility
volatility = np.std(log_returns) * np.sqrt(252)
volatility

**Note:** Risk-free rate data (3-month treasury yield for both USD and CNY) should be manually added below or retrieved via a suitable financial data API. Assume here:  
- USD 3-month rate: 5.00%  
- CNY 3-month rate: 2.00%

In [ ]:
# Autocorrelation plot
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(log_returns, lags=30)
plt.title('Autocorrelation of Log Returns')
plt.show()

In [ ]:
# Parameters for option pricing
S = data['Close'][-1]  # Current price
K = S                  # ATM option
T = (datetime.datetime(2025, 5, 31) - datetime.datetime.today()).days / 365
r = 0.05  # USD rate
q = 0.02  # CNY rate
sigma = volatility
T, S, K, r, q, sigma

In [ ]:
# CRR binomial model
def CRR_option_price(S, K, T, r, sigma, N=100, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r - q) * dt) - d) / (u - d)
    disc = np.exp(-r * dt)
    
    # Initialize asset prices at maturity
    ST = np.array([S * (u**j) * (d**(N-j)) for j in range(N+1)])
    if option_type == 'call':
        option_values = np.maximum(ST - K, 0)
    else:
        option_values = np.maximum(K - ST, 0)
    
    # Backward induction
    for i in range(N-1, -1, -1):
        option_values = disc * (p * option_values[1:] + (1-p) * option_values[:-1])
    
    return option_values[0]

call_crr = CRR_option_price(S, K, T, r, sigma, option_type='call')
put_crr = CRR_option_price(S, K, T, r, sigma, option_type='put')
call_crr, put_crr

In [ ]:
# Black-Scholes for currency option
def black_scholes_currency(S, K, T, r, q, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp(-q*T) * norm.cdf(-d1)

call_bs = black_scholes_currency(S, K, T, r, q, sigma, 'call')
put_bs = black_scholes_currency(S, K, T, r, q, sigma, 'put')
call_bs, put_bs

In [ ]:
# Monte Carlo Simulation
def monte_carlo_option_price(S, K, T, r, q, sigma, num_sim=10000, option_type='call'):
    np.random.seed(42)
    Z = np.random.standard_normal(num_sim)
    ST = S * np.exp((r - q - 0.5 * sigma**2)*T + sigma * np.sqrt(T) * Z)
    if option_type == 'call':
        payoffs = np.maximum(ST - K, 0)
    else:
        payoffs = np.maximum(K - ST, 0)
    return np.exp(-r*T) * np.mean(payoffs)

call_mc = monte_carlo_option_price(S, K, T, r, q, sigma, option_type='call')
put_mc = monte_carlo_option_price(S, K, T, r, q, sigma, option_type='put')
call_mc, put_mc

In [ ]:
# Comparison Table
methods = ['CRR', 'Black-Scholes', 'Monte Carlo']
call_prices = [call_crr, call_bs, call_mc]
put_prices = [put_crr, put_bs, put_mc]

summary_df = pd.DataFrame({'Method': methods, 'Call Price': call_prices, 'Put Price': put_prices})
summary_df